In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [5]:
data_dir = "/mnt/c/Users/LP031/onedrive_unige/data/immobiliare.it"
# Load data from CSV
df = pd.concat([
    pd.read_csv(f'{data_dir}/ads_savona_rent.csv'),
    pd.read_csv(f'{data_dir}/ads_genova_rent.csv')
], ignore_index=True)


In [7]:
df.columns

Index(['id', 'uuid', 'title', 'url', 'geoHash', 'type', 'typology_id',
       'typology_name', 'contract', 'isNew', 'luxury', 'visibility',
       'isProjectLike', 'isMosaic', 'propertiesCount', 'price_value',
       'price_formatted', 'price_min', 'price_max', 'price_range',
       'price_visible', 'surface', 'rooms', 'bathrooms', 'floor',
       'floor_number', 'floor_ga4value', 'elevator', 'address', 'latitude',
       'longitude', 'city', 'province', 'region', 'macrozone', 'nation',
       'description', 'caption', 'ga4features', 'ga4Heating', 'ga4Garage',
       'views', 'agency_id', 'agency_type', 'agency_name', 'agency_label',
       'agency_url', 'photo_id', 'photo_caption', 'photo_url_small',
       'photo_url_medium', 'photo_url_large', 'typologyGA4Translation',
       'matchSearch'],
      dtype='object')

In [8]:
df.dtypes

id                          int64
uuid                       object
title                      object
url                        object
geoHash                    object
type                       object
typology_id                 int64
typology_name              object
contract                   object
isNew                        bool
luxury                       bool
visibility                 object
isProjectLike                bool
isMosaic                     bool
propertiesCount           float64
price_value               float64
price_formatted            object
price_min                 float64
price_max                 float64
price_range                object
price_visible                bool
surface                    object
rooms                      object
bathrooms                  object
floor                      object
floor_number               object
floor_ga4value             object
elevator                   object
address                    object
latitude      

In [ ]:

# Basic cleaning and trimming
df['zone'] = df['zone'].str.strip()

# Convert strings to numeric where needed
df['surface'] = pd.to_numeric(df['surface'], errors='coerce')

# Encode categorical columns
encoder = LabelEncoder()
df['zone_encoded'] = encoder.fit_transform(df['zone'])
df['elevator_encoded'] = encoder.fit_transform(df['elevator'].astype(str))
df['balcony_encoded'] = encoder.fit_transform(df['balcony'].astype(str))
df['garage_encoded'] = encoder.fit_transform(df['garage'].astype(str))

# Define features and target
X = df[['zone_encoded', 'floor', 'elevator_encoded', 'balcony_encoded',
    'garage_encoded', 'surface']]
y = df['price']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
print("Linear Regression Test Score:", lr.score(X_test, y_test))

# RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print("Random Forest Test Score:", rf.score(X_test, y_test))

In [11]:
# Let's select a subset of relevant features for our ML model
# We'll focus on physical property attributes, location data, and price

# First, examine missing values to help with feature selection
print("Missing values per column:")
print(df.isnull().sum().sort_values(ascending=False).head(15))

# Create a list of features we think are important for predicting property prices
relevant_columns = [
    # Target variable
    'price_value',
    
    # Core property attributes
    'surface',          # Size of the property
    'rooms',            # Number of rooms
    'bathrooms',        # Number of bathrooms
    'floor',            # Floor level
    'typology_name',    # Type of property (apartment, house, etc.)
    
    # Amenities/features
    'elevator',         # Presence of elevator
    'ga4Heating',       # Heating type
    'ga4Garage',        # Garage availability
    'ga4features',      # Additional features (e.g., garden, pool, etc.)
    
    # Location data (important for real estate)
    'latitude',
    'longitude',
    'city',
    'macrozone',        # Area within the city
    'zone',             # More specific location
    
    # Additional attributes that might influence price
    'isNew',            # New construction or not
    'luxury'            # Luxury property flag
]

# Check which columns from our selection actually exist in the dataset
existing_columns = [col for col in relevant_columns if col in df.columns]
missing_columns = [col for col in relevant_columns if col not in df.columns]

print("\nSelected columns that exist in the dataset:")
print(existing_columns)

print("\nSelected columns missing from the dataset:")
print(missing_columns)

# Create our dataset with only the relevant columns that exist
if 'zone' not in df.columns and 'macrozone' in df.columns:
    # If zone doesn't exist but macrozone does, use macrozone
    print("\nUsing 'macrozone' instead of 'zone' for location information")
    df['zone'] = df['macrozone']

# Create the subset dataframe
df_subset = df[existing_columns].copy()

# Display the first few rows of our subset
print("\nSubset DataFrame:")
df_subset.head()

Missing values per column:
matchSearch        1347
propertiesCount    1347
price_max          1347
price_min          1347
description        1345
ga4Garage           976
views               767
elevator            657
price_value         517
caption             387
macrozone           297
photo_caption       280
agency_name         214
agency_id           214
agency_type         214
dtype: int64

Selected columns that exist in the dataset:
['price_value', 'surface', 'rooms', 'bathrooms', 'floor', 'typology_name', 'elevator', 'ga4Heating', 'ga4Garage', 'ga4features', 'latitude', 'longitude', 'city', 'macrozone', 'isNew', 'luxury']

Selected columns missing from the dataset:
['zone']

Using 'macrozone' instead of 'zone' for location information

Subset DataFrame:


,price_value,surface,rooms,bathrooms,floor,typology_name,elevator,ga4Heating,ga4Garage,ga4features,latitude,longitude,city,macrozone,isNew,luxury
0,NaN,65 m²,3,1,2°,Appartamento,NaN,Autonomo,1 in parcheggio/garage comune,"porta blindata, esposizione esterna, impianto ...",44.3601,8.5790,Varazze,Centro,False,False
1,2000.0,76 m²,3,1,Piano terra,Appartamento,NaN,Autonomo,1 in parcheggio/garage comune,"cancello elettrico, accesso per disabili, arma...",44.3456,8.5514,Celle Ligure,Celle Ligure Pecorile,False,False
2,1500.0,147 m²,5,2,"6°, con ascensore",Appartamento,True,Centralizzato,NaN,"cancello elettrico, porta blindata, esposizion...",44.3102,8.4807,Savona,Centro Storico,False,False
3,3500.0,44 m²,2,1,1°,Appartamento,NaN,Autonomo,1 in box privato/box in garage,"porta blindata, terrazzo, impianto tv centrali...",44.3516,8.5654,Varazze,Centro,False,False
4,NaN,50 m²,3,1,"2°, con ascensore",Appartamento,True,Autonomo,NaN,"videocitofono, impianto di allarme, esposizion...",44.0137,8.1801,Alassio,Regione Paradiso,False,False


In [14]:
df['floor_number'].unique().tolist()

['2',
 'piano terra',
 '6',
 '1',
 '4',
 nan,
 '3',
 'piano rialzato',
 'interrato (-1), piano terra, 1',
 '3 piano',
 'da seminterrato a piano terra',
 '1 piano',
 '5',
 'piano terra, 2',
 '4 piano',
 '2 piano',
 'da 2 a 3',
 'seminterrato, 4',
 '10',
 '8',
 '5 piano',
 '9',
 'piano terra, piano rialzato, 1',
 'su più livelli',
 'piano terra, 1',
 '7',
 'seminterrato',
 '1, 3',
 'ammezzato',
 'piano terra, da 1 a 3',
 'piano terra, da 1 a 2',
 'piano terra, piano rialzato',
 'seminterrato, 2',
 '4, 7',
 'seminterrato, 1',
 '6 piano',
 'interrato (-3)',
 'interrato (-1), 3',
 'interrato (-3), piano terra',
 'da 4 a 5',
 'piano rialzato, 25',
 'interrato (-1), piano terra',
 '3, 5',
 'piano rialzato, 1',
 '11']

# Floor Number Normalization

The `floor_number` column contains a mix of formats, including:
- Simple numeric values ('1', '2', '3', etc.)
- Italian text descriptions ('piano terra', 'piano rialzato')
- Floor ranges ('da 2 a 3', 'da seminterrato a piano terra')
- Multi-level properties ('su più livelli', 'piano terra, 1')

We'll normalize these values by converting them into numeric values, where:
- -3, -2, -1: Underground floors (interrato)
- 0: Ground floor (piano terra, seminterrato)
- 0.5: Mezzanine/raised ground floor (piano rialzato, ammezzato)
- 1-25: Standard floors
- For ranges or multiple floors, we'll use the highest floor as it typically represents the property's main position

In [19]:
# Create a function to normalize floor numbers
import re

def normalize_floor(floor_str):
    """
    Normalize floor number strings to numeric values.
    
    Args:
        floor_str: String representation of floor(s)
        
    Returns:
        float: Normalized floor number
        
    Examples:
        '3' -> 3.0
        'piano terra' -> 0.0
        'da 2 a 5' -> 5.0 (taking the highest value)
        'seminterrato, 2' -> 2.0 (taking the highest value)
    """
    if pd.isna(floor_str):
        return np.nan
    
    floor_str = str(floor_str).lower().strip()
    
    # Direct mapping for common Italian floor descriptions
    floor_mapping = {
        'piano terra': 0,
        'seminterrato': 0,  # Semi-basement, but usually at ground level
        'interrato (-1)': -1,
        'interrato (-2)': -2,
        'interrato (-3)': -3,
        'piano rialzato': 0.5,  # Raised ground floor
        'ammezzato': 0.5,  # Mezzanine
        'su più livelli': np.nan  # Multi-level, can't determine primary floor
    }
    
    # Check direct mapping first
    if floor_str in floor_mapping:
        return float(floor_mapping[floor_str])
    
    # For complex strings, extract all numbers
    numbers = []
    
    # Look for Italian range pattern "da X a Y"
    range_match = re.search(r'da\s+(\d+|\w+)\s+a\s+(\d+|\w+)', floor_str)
    if range_match:
        # Extract the second number (end of range)
        end_value = range_match.group(2)
        if end_value.isdigit():
            numbers.append(int(end_value))
        # If the end value is a word (like "piano terra"), map it
        elif end_value in floor_mapping:
            numbers.append(floor_mapping[end_value])
    
    # Extract all digits
    digit_matches = re.findall(r'\d+', floor_str)
    numbers.extend([int(d) for d in digit_matches])
    
    # Extract all known floor types
    for term, value in floor_mapping.items():
        if term in floor_str:
            numbers.append(value)
    
    # Return the highest floor (most relevant for pricing)
    if numbers:
        return float(max(numbers))
    
    # If no numbers found, return NaN
    return np.nan

# Test the function on our unique values
floor_test_df = pd.DataFrame({'floor_number': df['floor_number'].unique()})
floor_test_df['normalized_floor'] = floor_test_df['floor_number'].apply(normalize_floor)

# Display the results to verify the normalization
floor_test_df.sort_values('normalized_floor').head(20)

,floor_number,normalized_floor
36,interrato (-3),-3.0
1,piano terra,0.0
26,seminterrato,0.0
10,da seminterrato a piano terra,0.0
28,ammezzato,0.5
31,"piano terra, piano rialzato",0.5
7,piano rialzato,0.5
22,"piano terra, piano rialzato, 1",1.0
11,1 piano,1.0
43,"piano rialzato, 1",1.0


In [ ]:
# Perform more comprehensive data preprocessing on our subset

# Handle missing values and data conversion
df_clean = df_subset.copy()

# Convert string columns to appropriate numeric types
for col in ['surface', 'rooms', 'bathrooms']:
    if col in df_clean.columns:
        # Extract numbers from strings if necessary and convert to float
        if df_clean[col].dtype == 'object':
            df_clean[col] = df_clean[col].str.extract('(\d+\.?\d*)').astype(float)

# Handle floor information using our normalize_floor function
if 'floor_number' in df_clean.columns:
    # Apply our normalized floor function
    df_clean['floor_numeric'] = df_clean['floor_number'].apply(normalize_floor)
    
    # Drop the original floor column and keep the numeric version
    df_clean.drop('floor_number', axis=1, inplace=True)
elif 'floor' in df_clean.columns:
    # Apply our normalized floor function
    df_clean['floor_numeric'] = df_clean['floor'].apply(normalize_floor)
    
    # Drop the original floor column and keep the numeric version
    df_clean.drop('floor', axis=1, inplace=True)

# Handle categorical features
categorical_cols = ['typology_name', 'city', 'zone', 'elevator', 'ga4Heating', 'ga4Garage']
categorical_cols = [col for col in categorical_cols if col in df_clean.columns]

# Encode categorical variables
for col in categorical_cols:
    if df_clean[col].dtype == 'object':
        # Fill missing values with a placeholder
        df_clean[col] = df_clean[col].fillna('Unknown')
        
        # Create dummies with drop_first=True to avoid multicollinearity
        dummies = pd.get_dummies(df_clean[col], prefix=col, drop_first=True)
        
        # Add dummies to dataframe and drop the original column
        df_clean = pd.concat([df_clean, dummies], axis=1)
        df_clean.drop(col, axis=1, inplace=True)

# Handle boolean features
bool_cols = ['isNew', 'luxury']
bool_cols = [col for col in bool_cols if col in df_clean.columns]

for col in bool_cols:
    df_clean[col] = df_clean[col].astype(int)

# Drop rows with missing target values
df_clean = df_clean.dropna(subset=['price_value'])

# Display the processed dataset
print("Processed dataset shape:", df_clean.shape)
print("\nProcessed dataset columns:")
print(df_clean.columns.tolist())
print("\nMissing values in processed dataset:")
print(df_clean.isnull().sum().sum())

# Preview the processed dataset
df_clean.head()

In [ ]:
# Build and evaluate ML models using our cleaned dataset

# Drop rows with any remaining NaN values for training
df_ml = df_clean.dropna()
print(f"Dataset shape after dropping NaN values: {df_ml.shape}")

# Separate features and target
X = df_ml.drop('price_value', axis=1)
y = df_ml['price_value']

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Define a function to evaluate models
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Fit model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"Mean Absolute Error: €{mae:.2f}")
    print(f"Root Mean Squared Error: €{rmse:.2f}")
    print(f"R² Score: {r2:.4f}")
    
    return model, mae, rmse, r2

# Train and evaluate multiple models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

# Scale the features for linear models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Linear models (using scaled data)
print("\n--- Linear Regression ---")
lr_model, lr_mae, lr_rmse, lr_r2 = evaluate_model(
    LinearRegression(), X_train_scaled, X_test_scaled, y_train, y_test
)

print("\n--- Ridge Regression ---")
ridge_model, ridge_mae, ridge_rmse, ridge_r2 = evaluate_model(
    Ridge(alpha=1.0), X_train_scaled, X_test_scaled, y_train, y_test
)

# Tree-based models (don't need scaling)
print("\n--- Random Forest ---")
rf_model, rf_mae, rf_rmse, rf_r2 = evaluate_model(
    RandomForestRegressor(n_estimators=100, random_state=42), X_train, X_test, y_train, y_test
)

print("\n--- Gradient Boosting ---")
gb_model, gb_mae, gb_rmse, gb_r2 = evaluate_model(
    GradientBoostingRegressor(n_estimators=100, random_state=42), X_train, X_test, y_train, y_test
)

# Compare model performance
models = ['Linear Regression', 'Ridge Regression', 'Random Forest', 'Gradient Boosting']
r2_scores = [lr_r2, ridge_r2, rf_r2, gb_r2]
mae_scores = [lr_mae, ridge_mae, rf_mae, gb_mae]

# Find best model based on R² score
best_model_idx = np.argmax(r2_scores)
print(f"\nBest model based on R² score: {models[best_model_idx]} (R² = {r2_scores[best_model_idx]:.4f})")

# If we have RandomForest or GradientBoosting, check feature importance
if best_model_idx >= 2:  # Index 2 or 3 (tree-based models)
    best_model = [lr_model, ridge_model, rf_model, gb_model][best_model_idx]
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nFeature Importance:")
    print(feature_importance.head(10))